This notebook prepare the catalogues that will be analysed by CIGALE for SED fitting and physical parameter estimation.

In [1]:
import numpy as np

from astropy.table import Table

from herschelhelp.filters import correct_galactic_extinction
from herschelhelp.external import convert_table_for_cigale

In [2]:
master_catalogue = Table.read("../../dmu32/dmu32_ELAIS-N1/data/ELAIS-N1_20170816.fits")

In [3]:
len(master_catalogue)

4026292

# Best sources

In this catalogue, qui keep the sources with good fluxes in **all** MIPS and SPIRE bands.

In [4]:
best_sources = (
    ~np.isnan(master_catalogue['f_mips_24']) & 
    ~master_catalogue['flag_mips_24'] & 
    ~np.isnan(master_catalogue['f_spire_250']) & 
    ~master_catalogue['flag_spire_250'] & 
    ~np.isnan(master_catalogue['f_spire_350']) & 
    ~master_catalogue['flag_spire_350'] & \
    ~np.isnan(master_catalogue['f_spire_500']) & 
    ~master_catalogue['flag_spire_500'] & 
    ~np.isnan(master_catalogue['redshift'])
)

In [5]:
best_catalogue = master_catalogue[best_sources].copy()

In [6]:
len(best_catalogue)

50271

In [7]:
# Correction for galactic extinction
best_catalogue = correct_galactic_extinction(best_catalogue, inplace=True)

In [8]:
# Convertion to CIGALE format
best_catalogue = convert_table_for_cigale(best_catalogue, inplace=True)

/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/numpy/ma/core.py:852: RuntimeWarning: invalid value encountered in greater_equal
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)
/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/astropy/table/column.py:1096: MaskedArrayFutureWarning: setting an item on a masked array which has a shared mask will not copy the mask and also change the original mask array in the future.
Check the NumPy 1.11 release notes for more information.
  ma.MaskedArray.__setitem__(self, index, value)
/opt/anaconda3/envs/herschelhelp_internal/lib/python3.6/site-packages/numpy/ma/core.py:852: RuntimeWarning: invalid value encountered in multiply
  return umath.absolute(a) * self.tolerance >= umath.absolute(b)


In [9]:
best_catalogue.write("data_tmp/ELAIS-N1_cigale_best_extcor_20170816.fits")